In [1]:
import pickle

from itertools import product
import sys
sys.path.append("../src")  # Adjust this path as needed
from denoising_methods import *

# Load data

In [2]:
with open("../data/eeg_datasets.pkl", "rb") as f:
    eeg_datasets_dict = pickle.load(f)

original_datasets = eeg_datasets_dict["datasets_mne_format"]
noisy_datasets = eeg_datasets_dict["datasets_artnoise_mne_format"]

# Perform denoising

### Wavelet Transform with Thresholding

#### Preparing parameters

In [3]:
wavelet_types = ['haar', 'db2', 'db6', 'db8', 'sym4', 'sym6']
levels = [1, 2, 3, 4, 5]
threshold_modes = ['soft', 'hard']

param_combinations = product(wavelet_types, levels, threshold_modes)

In [4]:
results_wt = []

In [5]:
for original, noisy in zip(original_datasets, noisy_datasets):
    signal, times = noisy[:]
    denoised_dict = {
        "dataset_original": original, # TODO change to just the name of dataset
        "dataset_noisy": noisy,
        "parameter_sets": [],
        "denoised_signals": [],
    }
    for wavelet_type, level, threshold_mode in param_combinations:
        params = {
            "wavelet_type": wavelet_type,
            "level": level,
            "threshold_mode": threshold_mode
        }
        denoised_dict["parameter_sets"].append(params)
        denoised_signal = denoise_multichannel(signal, wavelet_type, level, threshold_mode)
        print(denoised_signal.shape, signal.shape)
        denoised_dict["denoised_signals"].append(denoised_signal)
    results_wt.append(denoised_dict)


(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64, 60000)
(64, 60000) (64,

In [ ]:
with open("../data/denoised_data.pkl", "wb") as f:
    pickle.dump({"results_wavelet_denoising": results_wt}, f) # TODO change it cus shit is too big (1.8 GB)